In [52]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from crewai_tools import tool, BaseTool,FileReadTool
from tempfile import TemporaryDirectory

from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.utilities import GoogleSerperAPIWrapper

import os
import requests

In [53]:
os.environ["OPENAI_API_KEY"] = ""
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""


os.environ["SERPER_API_KEY"] = ""

In [54]:
llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")

In [55]:
# Tools

human_tool = load_tools(["human"])

# Google Search

search = GoogleSearchAPIWrapper()
search_images = GoogleSerperAPIWrapper(type="images")

google_search_tool = Tool(
    name = "google_search",
    description = "Search Google for files needed for {topic}. ",
    func = search.run,
)

google_images_tool = Tool(
    name = "google_search_images",
    description = "Search Google for images needed for {topic}. ",
    func = search_images.run,
)

# Website Requests tool
request_tool = load_tools(["requests_get"], allow_dangerous_tools = True)

# Image Creation Tool
image_tool = load_tools(["dalle-image-generator"])

@tool("download_tool")
def download_tool(url: str, file_name: str) -> str:
    """Give the url and the name the file should download as with the file ending."""

    response = requests.get(url)
    with open(f"{file_name}", "wb") as file:
    # Write the content of the response (image) to the file
        file.write(response.content)

    return f"File downloaded as {file_name}"

# File System Tools
# We'll make a temporary directory to avoid clutter
#working_directory = TemporaryDirectory()

# toolkit = FileManagementToolkit(
#     #root_dir=str(working_directory.name)
# )  # If you don't provide a root_dir, operations will default to the current working directory
# toolkit.get_tools()

# file_tools = FileManagementToolkit(
#     root_dir=str("~/home/william-chandler/Research/crewai"),
#     selected_tools=["read_file", "copy_file"],
# ).get_tools()

file_tool = [FileReadTool()]

# Works as a way to make a tool but isn't compatible with load_tools

# def image_download(image_url: str, image_name: str) -> str:
#     response = requests.get(image_url)
#     with open(f"{image_name}", "wb") as file:
#     # Write the content of the response (image) to the file
#         file.write(response.content)

#     return

# image_download_tool2 = Tool(
#     name = "Image_Download_Tool",
#     func = image_download,
#     description = "Give the url of the image and the name the image should download as with the file ending."
# )


In [56]:
# Manager Agents

manager = Agent(
  name = "Manager",
  #llm = llm,
  role='Senior_Manager',
  goal = (
    'Over see the creation of {topic}. '
    'Plan how to create {topic}. '
    'Organize the tasks that must be completed. '
    'Delegate these tasks to co-workers. '
    'Make sure each round co-workers work together. '
    'Make sure round one co-workers work first, then round two co-workers work second, then round three co-workers work third. '
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  #tools = human_tool
  allow_delegation = True,
)

In [57]:
# Round One Agents
round_one_programmer = Agent(
  name = "round_one_programmer",
  #llm = llm,
  role='round_one_programmer',
  goal= (
    'Write perfect python code to create {topic}. '
    'Resolve any bugs or errors in the code. '
    #"Always start working before the round two co-workers. "
    #"Never finalize your answer without delegating the task of testing the code to the round_one_tester and implementing the changes requested by the round_one_tester. "
    #"Always begin writing code before delegating the task of testing the code to the round_one_tester. "
    #"When using the Action: Delegate to co-worker always specify the coworker: round_one_tester. "
    "When using the Action: Ask question to co-worker always specify the coworker: round_one_tester. "
  ),
  verbose=True,
  memory=True,
  backstory=(
    "You create and write perfect python code. "
  ),
  #tools=file_tools,
  allow_delegation = False
)

round_one_bug_fixer = Agent(
    name = "round_one_bug_fixer",
    role = "round_one_bug_fixer",
    goal = (
        "Code the changes to the round_one_programmer's code suggested by the round_one_tester. "
        "Do not make comments about where the changes should go like this: '# TODO: Implement the changes suggested by the round_one_tester # 1. Improve collision detection # 2. Optimize performance # 3. Enhance user interface # 4. Improve input handling # 5. Implement error handling'. "
        
        #"Never finalize your answer without delegating the task of testing the code to the round_one_tester and implementing the changes requested by the round_one_tester. "
        #"Use the list of bugs and errors provided by the round_one_tester to edit the round_one_programmer's code. "
        "When using the Action: Delegate to co-worker always specify the coworker: round_one_tester. "
    ),
    tools = file_tool,
    verbose = True,
    memory = True,
    backstory = (
        "You are a team player who always always works with their co-workers and loves to take feedback from game testers. "
    ),
    allow_delegation = True
)

# Creating a writer agent with custom tools and delegation capability
round_one_tester = Agent(
  name = "round_one_tester",
  #llm = llm,
  role='round_one_tester',
  goal= (
      "To help the round_one_bug_fixer fix and update the round_one_programmer's code. "

    # 'Give feedback on {topic}, and ensure that the round_one_programmer knows of any buggs or errors. '
    # 'Work with round_one_bug_fixer. '
    # 'Work before the round two and round three co-workers. '
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Dedicated game tester. "
  ),
  #tools= human_tool,
  allow_delegation = False
)

In [58]:
# Round Two Agents

round_two_programmer = Agent(
  name = "round_two_programmer",
  #llm = llm,
  role='Round_Two_Senior_Coder',
  goal= (
    'Write perfect python code to create {topic}. '
    'Resolve any bugs or errors in the code. '
    #"Always start working after the round two co-workers. "
    #"Always begin writing code before delegating the task of testing the code to the round_two_tester. "
    #"Never finalize your answer without delegating the task of testing the code to the round_two_tester and implementing the changes requested by the round_two_tester. "
    #"When using the Action: Delegate to co-worker always specify the coworker: round_two_tester. "
    "When using the Action: Ask question to co-worker always specify the coworker: tound_two_tester. "
    "Work after Round One co-workers and before round three co-workers. " 
  ),
  verbose=True,
  memory=True,
  backstory=(
    "You create and write perfect python code. "
  ),
  #tools=file_tools,
  allow_delegation = False
)

round_two_bug_fixer = Agent(
    name = "round_two_bug_fixer",
    role = "Round_Two_Bug_Fixer",
    goal = (
        "Never finalize your answer without delegating the task of testing the code to the round_two_tester and implementing the changes requested by the round_two_tester. "
        "Use the list of bugs and errors provided by the round_two_tester to edit the round_two_programmer's code. "
        "When using the Action: Delegate to co-worker always specify the coworker: round_two_tester. "
    ),
    tools = file_tool,
    verbose = True,
    memory = True,
    backstory = (
        "You are a team player who always always works with their co-workers and loves to take feedback from game testers. "
    ),
    allow_delegation = True
)

# Creating a writer agent with custom tools and delegation capability
round_two_tester = Agent(
  name = "round_two_tester",
  #llm = llm,
  role='Round_Two_Tester',
  goal= (
    'Give feedback on {topic}, and ensure that the round_two_programmer knows of any buggs or errors. '
    'Work with round_two_programmer. '
    'Work after the round one co-workers and before the round three co-workers. '
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Dedicated game tester. "
  ),
  #tools= human_tool,
  allow_delegation = False
)

In [59]:
# Round Three Agents

round_three_programmer = Agent(
  name = "round_three_programmer",
  #llm = llm,
  role='Round_Three_Senior_Coder',
  goal= (
    'Write perfect python code to create {topic}. '
    'Resolve any bugs or errors in the code. '
    #"Always start working after the round two co-workers. "
    #"Never finalize your answer without delegating the task of testing the code to the round_three_tester and implementing the changes requested by the round_three_tester. "
    #"Always begin writing code before delegating the task of testing the code to the round_three_tester. "
    #"Work with the round_three_tester. "
    "Work after round one and round two co-workers. " 
  ),
  verbose=True,
  memory=True,
  backstory=(
    "You create and write perfect python code. "
  ),
  #tools=file_tools,
  allow_delegation = False
)

round_three_bug_fixer = Agent(
    name = "round_three_bug_fixer",
    role = "round_three_bug_fixer",
    goal = (
        "Code the changes to the round_one_programmer's code suggested by the round_three_tester. "
        "Do not make comments about where the changes should go like this: '# TODO: Implement the changes suggested by the round_one_tester # 1. Improve collision detection # 2. Optimize performance # 3. Enhance user interface # 4. Improve input handling # 5. Implement error handling'. "
        
        #"Never finalize your answer without delegating the task of testing the code to the round_one_tester and implementing the changes requested by the round_one_tester. "
        #"Use the list of bugs and errors provided by the round_one_tester to edit the round_one_programmer's code. "
        "When using the Action: Delegate to co-worker always specify the coworker: round_three_tester. "
    ),
    tools = file_tool,
    verbose = True,
    memory = True,
    backstory = (
        "You are a team player who always always works with their co-workers and loves to take feedback from game testers. "
    ),
    allow_delegation = True
)

# Creating a writer agent with custom tools and delegation capability
round_three_tester =Agent(
  name = "round_three_tester",
  #llm = llm,
  role='Round_Three_Tester',
  goal= (
    'Give feedback on {topic}, and ensure that the round_three_programmer knows of any buggs or errors. '
    'Work with round_three_programmer. '
    'Work after the round one co-workers and round two co-workers. '
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Dedicated game tester. "
  ),
  #tools= human_tool,
  allow_delegation = False
)

In [60]:
# Other Agents

reviewer = Agent(
    name = "Reviwer",
    role = 'Reviewer',
    goal = (
        "Ensure your coworkers continue working until the human is satisfied. "
        #"Ensure that all human feedback has been implemented and the code works as intended. "           #HUMAN FEEDBACK
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Dedicated to your job you love the human and will make sure your coworkers do their job. "
    ),
    #tools = human_tool,
    allow_delegation = True
)

feedback_coder = Agent(
    name = "Feedback_Implementer",
    role = "Feedback_Implementer",
    goal =(
        'You write perfect python code. '
        'Read and use the code written by the programmer in the file game_code. '
        'Update the code written by the programmer. '
        #'You compile code and updates to that code into a final code file.'
    ),
    verbose = True,
    memory = True,
    backstory = (
        "You always make sure that all bugs and errors are fixed. "
    ),
    tools = file_tool,
    allow_delegation = True
)
# # Creating artist agent

artist = Agent(
  name = "Artist",
  #llm = llm,
  role = "Artist",
  goal = (
    'Search Google for files to download. '
    'Download files from urls found from searching google. '
    'Download images created by the Artist to be used by the programmer. '
    'Give the programmer information on the contents and file path of each file. '
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Is an artist who always finds the best image. "
    "You love downloading files and naming them proper names. "
  ),
  tools = [image_tool[0], google_images_tool, request_tool[0], download_tool],
  allow_delegation = True
)

# downloader = Agent(
#     name = "Downloader",
#     role = "Downloader",
#     goal = (
#         'Search Google for files to download. '
#         'Download files from urls found from searching google. '
#         'Download images created by the Artist to be used by the programmer. '
#     ),
#     verbose = True,
#     memory = True,
#     backstory = (
#         "You love downloading files and naming them proper names. "
#     ),
#     tools = [google_tool, download_tool],
#     allow_delegation = False
# )

In [61]:
# Manager Task

manager_task = Task(
  description = (
    "Understand the task given. "
    "Plan the exection of the task, and delegate jobs to co-workers. "
  ),
  expected_output = (
    "A python program that runs {topic}. "
    "Do not finalize your answer until the human says the game is complete. "
    #"Ask the human for feed back on the game before finalizing your answer. "                                                              #HUMAN FEEDBACK
    # "Do not finalize your answer until the reviewer recieves confirmation from the human that the game is complete. "                     #HUMAN FEEDBACK
    "Do not finalize your answer until the reviewer confirms the code is ready. "                                                           
  ),
  agent = manager,
  human_input = False,
  async_execution = False,
  #output_file = "game.py",
)

In [62]:
# Round One Tasks
round_one_programmer_task = Task(
  description = (
    #"Never finalize your answer without delegating the task of testing the code to the round_one_tester and implementing the changes requested by the round_one_tester. "
    "Write a single file with perfect python code to create {topic}. "
    #"Delegate the task of testing the code to the round_one_tester and implement any changes requested before finalizing your answer. "
    #"Do not finalize your answer until the reviewer has recieved confirmation from the human the game is complete. "                      #HUMAN FEEDBACK
    #"Do not finalize your answer until the reviewer confirms the code is ready. "                                    
    #"Make sure to use the art from the artist in the code for {topic} "                                        
  ),
  expected_output=(
    "A single executable file that contains the code for {topic}. "
    #"The executive file should only contain the code for {topic}. "
  ),
  #tools=[search_tool],
  agent= round_one_programmer,
  human_input = False,
  async_execution = False,
  output_file = "round_one_game_code.py",
)

round_one_bug_fixer_task = Task(
    description = (
       #f"Read the code written by the round_one_programmer from the disk with file path {round_one_programmer_task.output_file}. "
       f"Delegate the task of testing the code to the round_one_tester, and implement the changes suggested to the code from the disk with file path {round_one_programmer_task.output_file}. "
       #"After round_one_tester tests the code make sure to fix the bugs and errors provided by the round_one_tester. " 
       #"Make the changes provided by the round_one_tester. " 
    ),
    expected_output = (
      "A single executable file that contains the code for {topic}. "
      #"Updated code with all changes suggested by the round_one_tester made. "  
    ),
    tools = file_tool,
    agent = round_one_bug_fixer,
    human_input = False,
    async_execution = False,
    output_file = "final_round_one_game_code.py"
)

# Writing task with language model configuration
round_one_tester_task = Task(
  description=(
    "Test the code created by the round_one_bug_fixer and report feedback including any bugs and errors. "
    "Your final answer should contain a list of the bugs and errors in the code and only the bugs and errors in the code. "
    # "Only ask the human for feedback on the game. Do not ask the human how to do anything. "                                                #HUMAN FEEDBACK
    # "Once you can find no more bugs or erros check with the human for feedback before finalizing your answer. "                             #HUMAN FEEDBACK
    # "Give the human feedback to your co-workers. "                                                                                           #HUMAN FEEDBACK
    # "Continue to request feedback from the human then give this feedback to the programmer until the human confirms the game is finished. "  #HUMAN FEEDBACK
  ),
  expected_output = 'A report to the round_two_programmer containing all the bugs and errors in the code and only the bugs and errors in the code.',
  #tools = human_tool,
  agent = round_one_tester,
  human_input = False,
  async_execution = False,
  output_file = 'round_one_errors.md' ,
)


In [63]:
# Round Two Tasks

round_two_programmer_task = Task(
  description=(
    #"Never finalize your answer without delegating the task of testing the code to the round_two_tester and implementing the changes requested by the round_two_tester. "
    #f"Read the code written by the round_one_bug_fixer from the disk with file path {round_one_bug_fixer_task.output_file}. "
    #f"Read the error report written by the round_one_tester from the disk with file path {round_one_tester_task.output_file}. "
    f"Make the changes from the disk with file path {round_one_tester_task.output_file} to the code from the disk with file path {round_one_bug_fixer_task.output_file}. "
    #"Read both the code written by the round_one_bug_fixer and the error report written by the round_one_tester. "
    #"Fix the bugs and errors in the code. "
    #"Implement changes to the code suggested by the round_one_tester. "
    #"Delegate the task of testing the code to the round_two_tester and implement any changes requested before finalizing your answer. "
    #"Do not finalize your answer until the reviewer has recieved confirmation from the human the game is complete. "                      #HUMAN FEEDBACK
    #"Do not finalize your answer until the reviewer confirms the code is ready. "                                    
    #"Make sure to use the art from the artist in the code for {topic} "                                        
  ),
  expected_output=(
    "A single executable file that contains the code for {topic}. "
    #"The executive file should only contain the code for {topic}. "
  ),
  tools= file_tool,
  agent= round_two_programmer,
  human_input = False,
  async_execution = False,
  output_file = "round_two_game_code.py",
)

round_two_bug_fixer_task = Task(
    description = (
       #f"Read the code written by the round_two_programmer from the disk with file path {round_two_programmer_task.output_file}. "
       "Delegate the task of testing the code to the round_two_tester. "
       #"After the round_two_tester tests the code make sure to fix the bugs and errors provided by the round_two_tester. " 
       "Make the changes provided by the round_two_tester. "
    ),
    expected_output = (
      "A single executable file that contains the code for {topic}. "  
    ),
    #tools = file_tool,
    agent = round_two_bug_fixer,
    human_input = False,
    async_execution = False,
    output_file = "final_round_two_game_code.py"
)

round_two_tester_task = Task(
  description=(
    "Test the code created by the round_two_bug_fixer and report feedback including any bugs and errors. "
    "Give feedback to the round_two_bug_fixer. "
    "Your final answer should contain a list of the bugs and errors in the code and only the bugs and errors in the code. "
    # "Only ask the human for feedback on the game. Do not ask the human how to do anything. "                                                #HUMAN FEEDBACK
    # "Once you can find no more bugs or erros check with the human for feedback before finalizing your answer. "                             #HUMAN FEEDBACK
    # "Give the human feedback to your co-workers. "                                                                                           #HUMAN FEEDBACK
    # "Continue to request feedback from the human then give this feedback to the programmer until the human confirms the game is finished. "  #HUMAN FEEDBACK
  ),
  expected_output = 'A report to the round_three_programmer containing all the bugs and errors in the code and only the bugs and errors in the code.',
  #tools = human_tool,
  agent = round_two_tester,
  human_input = False,
  async_execution = False,
  output_file = 'round_two_errors.md' ,
)

In [64]:
# Round Three Tasks

round_three_programmer_task = Task(
  description=(
    f"Read the code written by the round_two_bug_fixer from the disk with file path {round_two_bug_fixer_task.output_file}. "
    f"Read the error report written by the round_two_tester from the disk with file path {round_two_tester_task.output_file}. "
    "Implement changes to the round_two_bug_fixer's code suggested by the round_two_tester. "
    #"Delegate the task of testing the code to the round_three_tester and implement any changes requested before finalizing your answer. "

    #"Do not finalize your answer until the reviewer has recieved confirmation from the human the game is complete. "                      #HUMAN FEEDBACK
    #"Do not finalize your answer until the reviewer confirms the code is ready. "                                    
    #"Make sure to use the art from the artist in the code for {topic} "                                        
  ),
  expected_output=(
    "A single executable file that contains the code for {topic}. "
    #"The executive file should only contain the code for {topic}. "
  ),
  tools= file_tool,
  agent= round_three_programmer,
  human_input = False,
  async_execution = False,
  output_file = "round_three_game_code.py",
)

round_three_bug_fixer_task = Task(
    description = (
       #f"Read the code written by the round_two_programmer from the disk with file path {round_two_programmer_task.output_file}. "
       "Delegate the task of testing the code to the round_three_tester. "
       #"After the round_two_tester tests the code make sure to fix the bugs and errors provided by the round_two_tester. " 
       "Make the changes provided by the round_three_tester. "
    ),
    expected_output = (
      "A single executable file that contains the code for {topic}. "  
    ),
    #tools = file_tool,
    agent = round_two_bug_fixer,
    human_input = False,
    async_execution = False,
    output_file = "final_round_three_game_code.py"
)

round_three_tester_task = Task(
  description=(
    "Test the code created by the round_three_bug_fixer and report feedback including any bugs and errors. "
    "Give feedback to the round_three_bug_fixer. "
    "Your final answer should contain a list of the bugs and errors in the code and only the bugs and errors in the code. "
    # "Only ask the human for feedback on the game. Do not ask the human how to do anything. "                                                #HUMAN FEEDBACK
    # "Once you can find no more bugs or erros check with the human for feedback before finalizing your answer. "                             #HUMAN FEEDBACK
    # "Give the human feedback to your co-workers. "                                                                                           #HUMAN FEEDBACK
    # "Continue to request feedback from the human then give this feedback to the programmer until the human confirms the game is finished. "  #HUMAN FEEDBACK
  ),
  expected_output = 'A report to the round_three_programmer indicating any bugs or errors in the code.',
  #tools = human_tool,
  agent = round_three_tester,
  human_input = False,
  async_execution = False,
  output_file = 'round_three_errors.md' ,
)

In [65]:
# Other Tasks

reviewer_task = Task(
    description = (
        #"Make sure all human feedback has been implemented. "                                          #HUMAN FEEDBACK
        #"Make sure the code works as intended. "
        # "Request confirmation from the human on whether the game is complete. "                       #HUMAN FEEDBACK
        "Do not review the game until your co-workers believe they are done with their jobs. "
        "If the game is not complete tell the manager to implement any neccesarry changes. "
        "If the game is complete tell the programmer they may finalize their answer. "
    ),
    expected_output = (
        # 'A report to your co-workers indicating whether all the human feedback has been implemented. '
        'A report to your co-workers indicating whether the code is working as intended. '
    ),
    #tools = human_tool
    agent = reviewer,
    human_input = False,
    async_execution = False,
    output_file = "review.md",
)

feedback_coder_task = Task(
    description = (
        "Do not begin working until the programmer has finalized their answer. "
        f"Read the code written by the programmer from the disk with file path {round_two_programmer_task.output_file} "
        #"Take the code written by the programmer and add code to implement the all feedback. "                      #HUMAN FEEDBACK
        "Make sure to implement all changes suggested by the tester"
        #"Update this code to include all necessary features. "
        "Compile all the code written into a single code file. "
    ),
    expected_output = (
        #'Your final answer must be the full python code with all buggs and errors fixed and all human feedback implemented. ' #HUMAN FEEDBACK
        'Your final answer must be the full python code with all buggs and errors fixed. '
        'The final answer must only be the python code and nothing else. '
    ),
    tools = file_tool,
    agent = feedback_coder,
    human_input = False,
    async_execution = False,
    output_file = "final_game_code.py",
)

# Artist task
artist_task = Task(
  description = (
    "Search google for urls to files needed by the programmer. "
    "Download these files from the url. "
    "Make sure to input a good file name that the programmer will use to access the image. "
    "Generate images to be used in {topic}. "
    "Make sure these images are used in the game. "
  ),
  expected_output = "Generated Images to be used in {topic}",
  tools = [image_tool[0], google_images_tool, request_tool[0], download_tool],
  agent = artist,
  human_input = False,
  async_execution = False,
  output_file = 'image.md',
)

# # Downloader task
# downloader_task = Task(
#   description = (
#     "Search google for urls to files needed by the programmer. "
#     "Download these files from the url. "
#     "Make sure to input a good file name that the programmer will use to access the image. "
#     "Make sure these files are used in the game. "
#   ),
#   expected_output = "Downloaded files to be used in {topic}",
#   tools = [google_tool, download_tool],
#   agent = downloader,
#   human_input = False,
#   async_execution = False,
#   output_file = 'downloader.md'
# )

In [66]:
# # Crew Creation

# # Forming the tech-focused crew with some enhanced configurations
# game_crew = Crew(
#   agents = [manager, programmer, artist, tester, feedback_coder, reviewer],
#   tasks = [manager_task, programmer_task, artist_task, tester_task, feedback_coder_task, reviewer_task],
#   #agents = [programmer],
#   #tasks = [programmer_task],
#   process=Process.sequential,  # Optional: Sequential task execution is default
#   #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
#   verbose = False,
#   memory=True,
#   cache=True,
#   max_rpm=100,
#   share_crew=True,
#   step_callback = print("STEP ################################################"),
# )

In [67]:
test_crew = Crew(
  agents = [manager, artist, round_one_programmer, round_one_bug_fixer, round_one_tester, round_two_programmer, round_two_bug_fixer, round_two_tester],
  tasks = [manager_task, artist_task, round_one_programmer_task, round_one_bug_fixer_task, round_one_tester_task, round_two_programmer_task, round_two_bug_fixer_task, round_two_tester_task],
  #agents = [round_one_programmer],
  #tasks = [round_one_programmer_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = 2,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
)

2024-06-10 16:07:05,628 - 139299777345344 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


STEP ################################################


In [68]:
if os.path.exists('round_one_game_code.py'):
    os.remove("round_one_game_code.py")
if os.path.exists('final_round_one_mario.pnggame_code.py'):
    os.remove("final_round_one_game_code.py")
if os.path.exists('round_one_errors.md'):
    os.remove("round_one_errors.md")

    
result = test_crew.kickoff(inputs={'topic': 'The game Mario'})

print(result)
    

 [DEBUG]: == Working Agent: Senior_Manager
 [INFO]: == Starting Task: Understand the task given. Plan the exection of the task, and delegate jobs to co-workers. 


> Entering new CrewAgentExecutor chain...
To successfully oversee the creation of the game Mario, I need to break down the task into smaller, manageable subtasks and delegate them effectively to my co-workers. Here is the plan:

### Plan and Delegation:
1. **Concept and Design Phase (Round One)**
    - **Task 1:** Create the visual assets (sprites, backgrounds, etc.)
    - **Task 2:** Develop the initial code for the game mechanics (movement, jumping, collision detection, etc.)
    - **Task 3:** Fix any bugs in the initial code
    - **Task 4:** Test the initial game mechanics

2. **Advanced Coding and Debugging Phase (Round Two)**
    - **Task 5:** Refine and optimize the game code
    - **Task 6:** Fix any new bugs found during optimization
    - **Task 7:** Conduct thorough testing to ensure all features are working corre

2024-06-10 16:20:01,938 - 139296226936512 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--4015f58291242bc76493e60e3890a71c91b7f7b5e27389e27bff53b1d14e002e


I now can give a great answer.

Final Answer: Here is the complete, improved `mario_game.py` code with the necessary changes to improve collision detection, refine enemy interaction handling, and enhance item collection reliability:

```python
import pygame
import os

# Initialize Pygame
pygame.init()

# Screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Load images and sounds
MARIO_IMAGE = pygame.image.load(os.path.join('mario.png'))
ENEMY_IMAGE = pygame.image.load(os.path.join('enemy.png'))
ITEM_IMAGE = pygame.image.load(os.path.join('item.png'))
JUMP_SOUND = pygame.mixer.Sound(os.path.join('jump.wav'))
ITEM_COLLECT_SOUND = pygame.mixer.Sound(os.path.join('item_collect.wav'))
HIT_SOUND = pygame.mixer.Sound(os.path.join('hit.wav'))

# Mario settings
MARIO_WIDTH = 40
MARIO_HEIGHT = 60
GRAVITY = 1
JUMP_STRENGTH = 15

# Game classes
class Mario(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        se

2024-06-10 16:21:46,060 - 139296164021952 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--c407aae7758638e455bedaee47b1e1cdae4d88fe8a9ade24a02aec7fe46d8d32
